Preprocessing data

In [ ]:
!gdown https://drive.google.com/uc?id=1Lbk_pwATorDcUyk9VcGYzomLR4Rhj-lv #Downloading zip file from google drive
zip_path = '/content/Cat_GAN_1.zip' #Getting the path 
!unzip -q Cat_GAN_1.zip #Unzipping the folder
!rm Cat_GAN_1.zip #Removing the zip folder
import os #Imported required libraries for this step
import cv2
import numpy as np
os.mkdir("Training") #making the new Training and testing folders
os.mkdir("Testing")
dirs = ["Domino", "Stormy"] #Making the names for the subdirec
pardirs = ["/content/Training", "/content/Testing"]
for pardir in pardirs:
  for dir in dirs:
    path = os.path.join(pardir, dir)
    os.mkdir(path)
datadir1 = "/content/Cat_Classifier_Images_70-30 /Train/Domino"
datadir2 = "/content/Cat_Classifier_Images_70-30 /Train/Stormy"
datadir3 = "/content/Cat_Classifier_Images_70-30 /Test/Domino"
datadir4 = "/content/Cat_Classifier_Images_70-30 /Test/Stormy"
filelist1 = sorted(os.listdir(datadir1), key = lambda fname: int(fname.split("_")[0][-4:]))
filelist2 = sorted(os.listdir(datadir2), key = lambda fname: int(fname.split("_")[0][-4:]))
filelist3 = sorted(os.listdir(datadir3), key = lambda fname: int(fname.split("_")[0][-4:]))
filelist4 = sorted(os.listdir(datadir4), key = lambda fname: int(fname.split("_")[0][-4:]))
datadirs = [filelist1, filelist2, filelist3, filelist4]
inc = 0
idom = 0
istorm = 0
for filelist in datadirs:
  for fil in filelist:
    if inc == 0:
      path = "/content/Cat_Classifier_Images_70-30 /Train/Domino/" + fil
      idom += 1
      img = cv2.imread(path)
      imgResized = np.array((cv2.resize(img, (96, 128))))
      cv2.imwrite('/content/Training/Domino/DominoTR%03i.jpg' %idom, imgResized)
    elif inc == 1:
      path = "/content/Cat_Classifier_Images_70-30 /Train/Stormy/" + fil
      istorm += 1
      img = cv2.imread(path)
      imgResized = np.array((cv2.resize(img, (96, 128))))
      cv2.imwrite('/content/Training/Stormy/StormyTR%03i.jpg' %istorm, imgResized)
    elif inc == 2:
      path = "/content/Cat_Classifier_Images_70-30 /Test/Domino/" + fil
      img = cv2.imread(path)
      idom += 1
      imgResized = np.array((cv2.resize(img, (96, 128))))
      cv2.imwrite('/content/Testing/Domino/DominoTR%03i.jpg' %idom, imgResized)
    else:
      path = "/content/Cat_Classifier_Images_70-30 /Test/Stormy/" + fil
      img = cv2.imread(path)
      istorm += 1
      imgResized = np.array((cv2.resize(img, (96, 128))))
      cv2.imwrite('/content/Testing/Stormy/StormyTR%03i.jpg' %istorm, imgResized)
  inc += 1

Data augmentation (Bounding Box csv has to be mounted before running this cell)

In [ ]:
import numpy as np
from PIL import Image
import pandas as pd
datadirDom = '/content/Training/Domino'
datadirStorm = '/content/Training/Stormy'
datadirDom2 = '/content/Testing/Domino'
datadirStorm2 = '/content/Testing/Stormy'
datadirDom = os.listdir(datadirDom)
datadirStorm = os.listdir(datadirStorm)
datadirDom2 = os.listdir(datadirDom2)
datadirStorm2 = os.listdir(datadirStorm2)
data = pd.read_csv('/content/CatBoxRegression - Sheet1.csv')
labels = []
cat_images = []
Boxtrain = []
classes = []
def transformboxes(box, ref_hor, ref_vert):
  if ref_hor:
    box[0], box[2] = 1 - box[2], 1 - box[0]
  if ref_vert:
    box[1], box[3] = 1 - box[3], 1 - box[1]
  return box
i = 0
for fil in datadirDom:
  box = np.array((data['Startx'][i] / 96, data['Starty'][i] / 128, data['Endx'][i] / 96, data['Endy'][i] / 128))
  i += 1
  path = '/content/Training/Domino/' + fil
  image = Image.open(path)
  horz_image = np.array(image.transpose(method = Image.FLIP_LEFT_RIGHT))
  vert_image = np.array(image.transpose(method = Image.FLIP_TOP_BOTTOM))
  rot_image = np.array(image.rotate(180))
  image = np.array(image)
  cat_images.append(image)
  cat_images.append(horz_image)
  cat_images.append(vert_image)
  cat_images.append(rot_image)
  Boxtrain.append(box)
  Boxtrain.append(transformboxes(box, True, False))
  Boxtrain.append(transformboxes(box, False, True))
  Boxtrain.append(transformboxes(box, True, True))
  for j in range(4):
    classes.append(np.array((1, 0)))
for fil in datadirStorm:
  box = np.array((data['Startx'][i] / 96, data['Starty'][i] / 128, data['Endx'][i] / 96, data['Endy'][i] / 128))
  i += 1
  path = '/content/Training/Stormy/' + fil
  image = Image.open(path)
  horz_image = np.array(image.transpose(method = Image.FLIP_LEFT_RIGHT))
  vert_image = np.array(image.transpose(method = Image.FLIP_TOP_BOTTOM))
  rot_image = np.array(image.rotate(180))
  image = np.array(image)
  cat_images.append(image)
  cat_images.append(horz_image)
  cat_images.append(vert_image)
  cat_images.append(rot_image)
  Boxtrain.append(box)
  Boxtrain.append(transformboxes(box, True, False))
  Boxtrain.append(transformboxes(box, False, True))
  Boxtrain.append(transformboxes(box, True, True))
  for j in range(4):
    classes.append(np.array((0, 1)))
for fil in datadirDom2:
  box = np.array((data['Startx'][i] / 96, data['Starty'][i] / 128, data['Endx'][i] / 96, data['Endy'][i] / 128))
  i += 1
  path = '/content/Testing/Domino/' + fil
  image = Image.open(path)
  horz_image = np.array(image.transpose(method = Image.FLIP_LEFT_RIGHT))
  vert_image = np.array(image.transpose(method = Image.FLIP_TOP_BOTTOM))
  rot_image = np.array(image.rotate(180))
  image = np.array(image)
  cat_images.append(image)
  cat_images.append(horz_image)
  cat_images.append(vert_image)
  cat_images.append(rot_image)
  Boxtrain.append(box)
  Boxtrain.append(transformboxes(box, True, False))
  Boxtrain.append(transformboxes(box, False, True))
  Boxtrain.append(transformboxes(box, True, True))
  for j in range(4):
    classes.append(np.array((1, 0)))
for fil in datadirStorm2:
  box = np.array((data['Startx'][i] / 96, data['Starty'][i] / 128, data['Endx'][i] / 96, data['Endy'][i] / 128))
  i += 1
  path = '/content/Testing/Stormy/' + fil
  image = Image.open(path)
  horz_image = np.array(image.transpose(method = Image.FLIP_LEFT_RIGHT))
  vert_image = np.array(image.transpose(method = Image.FLIP_TOP_BOTTOM))
  rot_image = np.array(image.rotate(180))
  image = np.array(image)
  cat_images.append(image)
  cat_images.append(horz_image)
  cat_images.append(vert_image)
  cat_images.append(rot_image)
  Boxtrain.append(box)
  Boxtrain.append(transformboxes(box, True, False))
  Boxtrain.append(transformboxes(box, False, True))
  Boxtrain.append(transformboxes(box, True, True))
  for j in range(4):
    classes.append(np.array((0, 1)))
cat_images = np.array(cat_images)
Boxtrain = np.array((Boxtrain))
classes = np.array((classes))

Custom Convolutional Detector

In [ ]:
import numpy as np #Importing needed libraries
from tensorflow import random
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout, Conv2D, MaxPool2D, Flatten, Input
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras import optimizers
import matplotlib.pyplot as plt
seed = 1
np.random.seed(seed)
random.set_seed(seed)
X_train, X_test, y_train, y_test, z_train, z_test = train_test_split(cat_images, Boxtrain, classes, test_size = 0.3)
input = Input(shape = (128, 96, 3))
conv = Conv2D(128, (3, 3), input_shape = (128, 96, 3), activation = 'relu')(input)
conv = Conv2D(64, (3, 3), input_shape = (128, 96, 3), activation = 'relu')(conv)
conv = MaxPool2D(2, 2)(conv)
conv = Conv2D(64, (3, 3), input_shape = (128, 96, 3), activation = 'relu')(conv)
conv = Conv2D(64, (3, 3), input_shape = (128, 96, 3), activation = 'relu')(conv)
conv = MaxPool2D(2, 2)(conv)
conv = Flatten()(conv)
bbox = Dense(128, activation = 'relu')(conv)
bbox = Dropout(rate = 0.05)(bbox)
bbox = Dense(64, activation = 'relu')(bbox)
bbox = Dropout(rate = 0.05)(bbox)
bbox = Dense(64, activation = 'relu')(bbox)
bbox = Dropout(rate = 0.05)(bbox)
bbox = Dense(64, activation = 'relu')(bbox)
bbox = Dropout(rate = 0.05)(bbox)
bbox = Dense(64, activation = 'relu')(bbox)
bbox = Dropout(rate = 0.05)(bbox)
bbox = Dense(4, activation = 'sigmoid', name = 'Bbox_Fin')(bbox)
clas = Dense(128, activation = 'relu')(conv)
clas = Dropout(rate = 0.05)(clas)
clas = Dense(64, activation = 'relu')(clas)
clas = Dropout(rate = 0.05)(clas)
clas = Dense(64, activation = 'relu')(clas)
clas = Dropout(rate = 0.05)(clas)
clas = Dense(64, activation = 'relu')(clas)
clas = Dropout(rate = 0.05)(clas)
clas = Dense(64, activation = 'relu')(clas)
clas = Dropout(rate = 0.05)(clas)
clas = Dense(2, activation = 'sigmoid', name = 'Class_Fin')(clas)
detector = Model(inputs = input, outputs = [bbox, clas], name = 'Non-pretrained_Detector')
optimizer = optimizers.Adagrad(learning_rate = 1e-4)
detector.compile(optimizer = optimizer, loss = 'mse', metrics = ['accuracy'])
print (detector.summary())
epochs = [i for i in range(1000)]
history = detector.fit(X_train, [y_train, z_train], validation_data = (X_test, [y_test, z_test]), validation_steps = 5, epochs = 1000, steps_per_epoch = 20, batch_size = 40, verbose = False)
plt.scatter(epochs, history.history['val_loss'])
plt.ylabel('Validaiton loss')
plt.xlabel('Epochs')
plt.show()
plt.scatter(epochs, history.history['val_Bbox_Fin_accuracy'])
plt.ylabel("Bounding box Accuracy")
plt.xlabel("Epochs")
plt.show()
plt.scatter(epochs, history.history['val_Class_Fin_accuracy'])
plt.xlabel("Epochs")
plt.ylabel("Classifier Accuracy")
plt.show()
detector.save('MC_cat_detector.h5')

Using VGG16 Imagenet weights for cat bounding box detection

In [ ]:
from keras.applications import VGG16
import numpy as np #Importing needed libraries
from tensorflow import random
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Conv2D, MaxPool2D, Flatten, Input
from sklearn.model_selection import train_test_split
from keras import optimizers
from keras.models import Model
import matplotlib.pyplot as plt
seed = 1
np.random.seed(seed)
random.set_seed(seed)
X_train, X_test, y_train, y_test, z_train, z_test = train_test_split(cat_images, Boxtrain, classes, test_size = 0.3)
vgg = VGG16(weights = 'imagenet', include_top = False, input_tensor = Input(shape = (128, 96, 3)))
vgg.trainable = False
result = vgg.output
result = Flatten()(result)
clas = Dense(128, activation = 'relu')(result)
result = Dense(128, activation = 'relu')(result)
result = Dropout(rate = 0.1)(result)
result = Dense(64, activation = 'relu')(result)
result = Dropout(rate = 0.1)(result)
result = Dense(64, activation = 'relu')(result)
result = Dropout(rate = 0.1)(result)
result = Dense(64, activation = 'relu')(result)
result = Dropout(rate = 0.1)(result)
result = Dense(4, activation = 'sigmoid', name = 'Box_fin')(result)
clas = Dropout(rate = 0.1)(clas)
clas = Dense(64, activation = 'relu')(clas)
clas = Dropout(rate = 0.05)(clas)
clas = Dense(64, activation = 'relu')(clas)
clas = Dropout(rate = 0.05)(clas)
clas = Dense(64, activation = 'relu')(clas)
clas = Dropout(rate = 0.05)(clas)
clas = Dense(2, activation = 'sigmoid', name = 'Clas_fin')(clas)
detector = Model(inputs = vgg.input, outputs = [result, clas], name = 'Pretrained_ImageNet_Weights_Detector')
optimizer = optimizers.Adam(learning_rate = 1e-4)
detector.compile(optimizer = optimizer, loss = 'mse', metrics = ['accuracy'])
print (detector.summary())
epochs = [i for i in range(1000)]
historyy = detector.fit(X_train, [y_train, z_train], validation_data = (X_test, [y_test, z_test]), validation_steps = 5, epochs = 1000, steps_per_epoch = 20, batch_size = 40, verbose = False)
plt.scatter(epochs, historyy.history['val_loss'])
plt.ylabel('Validaiton loss')
plt.xlabel('Epochs')
plt.show()
plt.scatter(epochs, historyy.history['val_Box_fin_accuracy'])
plt.ylabel("Bounding box Validation Accuracy")
plt.xlabel("Epochs")
plt.show()
plt.scatter(epochs, historyy.history['val_Clas_fin_accuracy'])
plt.xlabel("Epochs")
plt.ylabel("Classifier Accuracy")
plt.show()
plt.scatter(epochs, historyy.history['Box_fin_accuracy'])
plt.ylabel("Bounding box Training Accuracy")
plt.xlabel("Epochs")
plt.show()
detector.save('MC_vgg16_cat_detector.h5')

Plotting Imagenet vs. non-Imagenet

In [ ]:
plt.scatter(epochs, historyy.history['val_loss'], label = 'Pretrained Conv')
plt.ylabel('Validaiton loss')
plt.xlabel('Epochs')
plt.scatter(epochs, history.history['val_loss'], label = 'Non-pretrained Conv')
plt.ylabel('Validaiton loss')
plt.xlabel('Epochs')
plt.legend()
plt.show()
plt.scatter(epochs, historyy.history['val_Box_fin_accuracy'], label = 'Pretrained Conv')
plt.ylabel("Bounding box Accuracy")
plt.xlabel("Epochs")
plt.scatter(epochs, history.history['val_Bbox_Fin_accuracy'], label = 'Non-pretrained Conv')
plt.ylabel("Bounding box Accuracy")
plt.xlabel("Epochs")
plt.legend()
plt.show()
plt.scatter(epochs, historyy.history['val_Clas_fin_accuracy'], label = 'Pretrained Conv')
plt.xlabel("Epochs")
plt.ylabel("Classifier Accuracy")
plt.scatter(epochs, history.history['val_Class_Fin_accuracy'], label = 'Non-pretrained Conv')
plt.xlabel("Epochs")
plt.ylabel("Classifier Accuracy")
plt.legend()
plt.show()

Bounding box prediction

In [ ]:
from keras.models import load_model
detector = load_model('/content/MC_cat_detector.h5')
image1 = np.array((cv2.resize(cv2.imread('/content/Testing/Domino/DominoTR128.jpg'), (96, 128))))
image2 = np.array((cv2.resize(cv2.imread('/content/Testing/Stormy/StormyTR117.jpg'), (96, 128))))
images = np.array((image1, image2))
boxes, classes = detector.predict(images)
def resize_box(box, h, w):
  i = 0
  boxx = []
  for b in box:
    if i % 2 == 0:
      b = b * w
      boxx.append(b)
    else:
      b = b * h
      boxx.append(b)
  return boxx
boxess = []
for box in boxes:
  box = resize_box(box, 128, 96)
  boxess.append(box)
i = 0
for image in images:
  plt.imshow(image)
  plt.show()
  print (boxess[i])
  i += 1